In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Chronic_obstructive_pulmonary_disease_(COPD)"
cohort = "GSE210272"

# Input paths
in_trait_dir = "../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)"
in_cohort_dir = "../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)/GSE210272"

# Output paths
out_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/GSE210272.csv"
out_gene_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/gene_data/GSE210272.csv"
out_clinical_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/clinical_data/GSE210272.csv"
json_path = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
```python
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains mRNA expression profiled using Affymetrix Human Gene 1.0 ST Arrays,
# which indicates it contains gene expression data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For COPD trait:
# Looking at the sample characteristics, we can infer COPD from FEV1 % predicted values in index 4.
# In COPD diagnosis, FEV1 % predicted < 80% is often used as a criterion, with lower values indicating more severe COPD.
# The values range from 15% to 69%, suggesting these are patients with varying degrees of COPD severity.
trait_row = 4

# For age:
# Age data is available at index 2
age_row = 2

# For gender:
# Gender data is available at index 1
gender_row = 1

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert FEV1 % predicted values to binary COPD classification.
    FEV1 < 50% indicates severe/very severe COPD (1), while FEV1 ≥ 50% indicates mild/moderate COPD (0)."""
    if value is None or ":" not in value:
        return None
    try:
        fev1_value = float(value.split(":")[1].strip())
        # Using 50% as a cutoff for severe COPD (1) vs. less severe COPD (0)
        return 1 if fev1_value < 50 else 0
    except (ValueError, IndexError):
        return None

def convert_age(value):
    """Convert age string to float."""
    if value is None or ":" not in value:
        return None
    try:
        return float(value.split(":")[1].strip())
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    """Convert gender string to binary (0 for Female, 1 for Male)."""
    if value is None or ":" not in value:
        return None
    gender = value.split(":")[1].strip().lower()
    if "female" in gender:
        return 0
    elif "male" in gender:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path, 
                             is_gene_available=is_gene_available, is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we proceed with clinical feature extraction
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    # First, we need to create this dictionary as it's shown in the previous output
    sample_char_dict = {
        0: ['original geo accession: GSM912197', 'original geo accession: GSM912198', 'original geo accession: GSM912199', 'original geo accession: GSM912200', 'original geo accession: GSM912201', 'original geo accession: GSM912202', 'original geo accession: GSM912203', 'original geo accession: GSM912204', 'original geo accession: GSM912205', 'original geo accession: GSM912206', 'original geo accession: GSM912207', 'original geo accession: GSM912208', 'original geo accession: GSM912209', 'original geo accession: GSM912210', 'original geo accession: GSM912211', 'original geo accession: GSM912212', 'original geo accession: GSM912213', 'original geo accession: GSM912214', 'original geo accession: GSM912215', 'original geo accession: GSM912216', 'original geo accession: GSM912217', 'original geo accession: GSM912218', 'original geo accession: GSM912219', 'original geo accession: GSM912220', 'original geo accession: GSM912221', 'original geo accession: GSM912222', 'original geo accession: GSM912223', 'original geo accession: GSM912224', 'original geo accession: GSM912225', 'original geo accession: GSM912226'],
        1: ['Sex: Male', 'Sex: Female'],
        2: ['age: 57.6', 'age: 61', 'age: 66.3', 'age: 71.5', 'age: 63.4', 'age: 50.3', 'age: 60.3', 'age: 66.6', 'age: 57', 'age: 68.9', 'age: 59.2', 'age: 66.9', 'age: 51.9', 'age: 63.7', 'age: 67.2', 'age: 62.3', 'age: 59.1', 'age: 66.2', 'age: 56.6', 'age: 65.1', 'age: 63.3', 'age: 61.3', 'age: 71.4', 'age: 60.4', 'age: 73.2', 'age: 67.8', 'age: 71.2', 'age: 62.7', 'age: 72.4', 'age: 68.8'],
        3: ['smoking status: Current smoker', 'smoking status: Former smoker'],
        4: ['fev1 % predicted: 15', 'fev1 % predicted: 20', 'fev1 % predicted: 31', 'fev1 % predicted: 35', 'fev1 % predicted: 37', 'fev1 % predicted: 38', 'fev1 % predicted: 39', 'fev1 % predicted: 40', 'fev1 % predicted: 41', 'fev1 % predicted: 45', 'fev1 % predicted: 46', 'fev1 % predicted: 48', 'fev1 % predicted: 49', 'fev1 % predicted: 50', 'fev1 % predicted: 51', 'fev1 % predicted: 52', 'fev1 % predicted: 54', 'fev1 % predicted: 55', 'fev1 % predicted: 57', 'fev1 % predicted: 58', 'fev1 % predicted: 59', 'fev1 % predicted: 61', 'fev1 % predicted: 62', 'fev1 % predicted: 63', 'fev1 % predicted: 64', 'fev1 % predicted: 65', 'fev1 % predicted: 66', 'fev1 % predicted: 67', 'fev1 % predicted: 68', 'fev1 % predicted: 69'],
        5: ['cancer status: NA']
    }
    
    # Convert this dictionary to a DataFrame format that can be used with geo_select_clinical_features
    # Create a DataFrame with the sample characteristics
    clinical_data = pd.DataFrame(sample_char_dict)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of extracted clinical features:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save to CSV
    selected_clinical_df.to


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import json
import os
import re
from typing import Optional, Dict, Any, Callable

# Load clinical data
input_files = os.listdir(in_cohort_dir)
clinical_data_file = None
for file in input_files:
    if file.endswith('_sample_characteristics.txt'):
        clinical_data_file = os.path.join(in_cohort_dir, file)
        break

if clinical_data_file:
    clinical_data = pd.read_csv(clinical_data_file, sep='\t')
    print("Sample characteristics loaded.")
    print(f"Shape: {clinical_data.shape}")
    sample_char_dict = clinical_data.to_dict(orient='list')
    unique_values = {key: list(set(val)) for key, val in sample_char_dict.items() if key != 'Sample'}
    print("Unique values in each column:")
    for key, values in unique_values.items():
        print(f"{key}: {values}")
else:
    print("No sample characteristics file found.")
    sample_char_dict = {}
    unique_values = {}

# Let's check if gene expression data is likely available
is_gene_available = True  # Default assumption for GEO datasets unless clear evidence suggests otherwise

# Function to extract value from cell content
def extract_value(cell):
    if isinstance(cell, str) and ':' in cell:
        return cell.split(':', 1)[1].strip()
    return cell

# Analyze what rows contain trait, age, and gender information
trait_row = None
age_row = None
gender_row = None

# Check available rows in sample characteristics
for row_id, values in unique_values.items():
    for value in values:
        if isinstance(value, str):
            value_lower = value.lower()
            
            # Check for COPD information
            if 'copd' in value_lower or 'chronic obstructive pulmonary disease' in value_lower:
                trait_row = row_id
            
            # Check for age information
            elif 'age' in value_lower:
                age_row = row_id
            
            # Check for gender/sex information
            elif 'gender' in value_lower or 'sex' in value_lower:
                gender_row = row_id

# Define conversion functions
def convert_trait(value):
    if value is None:
        return None
    
    value = extract_value(value)
    if not value:
        return None
    
    value_lower = value.lower()
    if 'copd' in value_lower or 'yes' in value_lower or 'patient' in value_lower:
        return 1
    elif 'control' in value_lower or 'no' in value_lower or 'healthy' in value_lower:
        return 0
    else:
        return None

def convert_age(value):
    if value is None:
        return None
    
    value = extract_value(value)
    if not value:
        return None
    
    # Try to extract a number from the value
    age_match = re.search(r'(\d+(?:\.\d+)?)', value)
    if age_match:
        return float(age_match.group(1))
    else:
        return None

def convert_gender(value):
    if value is None:
        return None
    
    value = extract_value(value)
    if not value:
        return None
    
    value_lower = value.lower()
    if 'female' in value_lower or 'f' == value_lower:
        return 0
    elif 'male' in value_lower or 'm' == value_lower:
        return 1
    else:
        return None

# Check if trait data is available
is_trait_available = trait_row is not None

# Save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# If clinical data is available, extract and save it
if is_trait_available:
    selected_clinical = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    # Preview the extracted clinical data
    preview = preview_df(selected_clinical)
    print("Clinical data preview:")
    print(preview)
    
    # Create the output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data
    selected_clinical.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print("No trait data available. Skipping clinical feature extraction.")


### Step 4: Gene Data Extraction

In [ ]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 5: Gene Identifier Review

In [ ]:
# Examining the gene identifiers from the previous step
# The identifiers are in the format ENSG00000000003_at, which are Ensembl gene IDs 
# (ENSG prefix) with an "_at" suffix, not standard human gene symbols

# Ensembl IDs need to be mapped to human gene symbols for better interpretability
# and consistency with other datasets
requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene annotation data from the SOFT file
gene_annotation = get_gene_annotation(soft_file)

# 2. Extract and process the gene expression data again to ensure it's properly loaded
gene_data = get_genetic_data(matrix_file)
print(f"Gene data shape: {gene_data.shape}")
print(f"Gene data index sample: {gene_data.index[:5].tolist()}")

# Create a mapping dataframe with the ID and ORF columns
mapping_df = gene_annotation[['ID', 'ORF']].copy()
print(f"Mapping dataframe shape: {mapping_df.shape}")

# Set the Gene column to be the Ensembl IDs (from ORF column)
# These will later be matched against gene_synonym.json during normalization
mapping_df['Gene'] = mapping_df['ORF']
print("First few rows of mapping dataframe:")
print(mapping_df.head())

# Check the overlap between gene data index and mapping IDs
common_ids = set(mapping_df['ID']) & set(gene_data.index)
print(f"Number of IDs in common between mapping and gene data: {len(common_ids)}")

# Create output directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# 3. Apply gene mapping to convert probe-level data to gene expression data
try:
    # Apply the mapping to convert probe IDs to gene symbols and aggregate
    gene_data = apply_gene_mapping(gene_data, mapping_df)
    print(f"Gene expression data converted to gene symbols. Shape: {gene_data.shape}")
    
    # Save the processed gene data
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
    
    # Preview the gene data
    print("\nPreview of gene expression data:")
    print(gene_data.head())
except Exception as e:
    print(f"Error applying gene mapping: {e}")
    import traceback
    traceback.print_exc()
    
    # As a fallback, save the original gene data
    print("Saving original gene data as fallback")
    gene_data.to_csv(out_gene_data_file)


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data 
# (This was already done in the previous step, so no need to repeat)
print(f"Normalized gene data shape: {gene_data.shape}")

# 2. Examine the sample IDs in the gene expression data to understand the structure
print("Gene data column names (sample IDs):")
print(gene_data.columns[:5])  # Print first 5 for brevity

# Inspect the clinical data format from the matrix file directly
background_info, clinical_data = get_background_and_clinical_data(matrix_file)
print("\nRaw clinical data structure:")
print(f"Clinical data shape: {clinical_data.shape}")
print(f"Clinical data columns: {clinical_data.columns[:5]}")

# Get the sample characteristics to re-extract the disease information
sample_characteristics_dict = get_unique_values_by_row(clinical_data)
print("\nSample characteristics dictionary:")
print(sample_characteristics_dict)

# 3. Directly create clinical features from the raw data again
# Verify trait row contains the disease information (OA vs RA)
print("\nValues in trait row:")
trait_values = clinical_data.iloc[trait_row].values
print(trait_values[:5])

# Create clinical dataframe with proper structure
# First get the sample IDs from gene data as these are our actual sample identifiers
sample_ids = gene_data.columns.tolist()

# Create the clinical features dataframe with those sample IDs
clinical_features = pd.DataFrame(index=[trait], columns=sample_ids)

# Fill the clinical features with our trait values by mapping GSM IDs to actual values
for col in clinical_data.columns:
    if col in sample_ids:
        # Extract the disease value and convert it
        disease_val = clinical_data.iloc[trait_row][col]
        clinical_features.loc[trait, col] = convert_trait(disease_val)

print("\nCreated clinical features dataframe:")
print(f"Shape: {clinical_features.shape}")
print(clinical_features.iloc[:, :5])  # Show first 5 columns

# 4. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
print(f"\nLinked data shape before handling missing values: {linked_data.shape}")

# 5. Handle missing values - we need to use the actual column name, not the trait variable
# First identify the actual trait column name in the linked data
trait_column = clinical_features.index[0]  # This should be 'Osteoarthritis'
print(f"Actual trait column in linked data: {trait_column}")

# Now handle missing values with the correct column name
linked_data_clean = handle_missing_values(linked_data, trait_column)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 6. Evaluate bias in trait and demographic features
is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait_column)

# 7. Conduct final quality validation
note = "Dataset contains gene expression data from synovial fibroblasts of RA and OA patients. Data includes high serum and low serum responses."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=(linked_data_clean.shape[0] > 0),
    is_biased=is_biased,
    df=linked_data_clean,
    note=note
)

# 8. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable due to quality issues - linked data not saved")